# Recommendation API

In [2]:
import pandas as pd
db = pd.read_csv("predictions total2.csv")
names = pd.read_csv("Artists.dat", sep="\t")
names.head()

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [3]:
# Results of the function
def recommended_artists(user_id, n_artists):
    # Assumes a format with columns names as artistId, rows as users' ratings, with the first column being the userId
    
    
    # Get the user's predictions, sorted. Only keeps the n artists best ranked.
    ratings = db[db.iloc[:,0]==user_id].iloc[0].sort_values(ascending=False).reset_index()[1:n_artists+1]

    # The artist IDs got turned into string inthe process
    ratings.iloc[:,0] = ratings.iloc[:,0].astype(int)

    # Fetches the artist names using their IDs
    artist_names = [names[names.iloc[:,0] == ratings.iloc[i,0] ].iloc[0, 1] for i in range(len(ratings))]

    return [{"id": ratings.iloc[i, 0], # ID retrieved in the ratings dataframe
      "name": artist_names[i], # Name retrieved in the artist names list defined above
      "score": ratings.iloc[i, 1]} # Score retrieved in the ratings dataframe
     for i in range(n_artists)]

recommended_artists(108, 5)

[{'id': 227, 'name': 'The Beatles', 'score': 9.13710199643455},
 {'id': 163, 'name': 'Pink Floyd', 'score': 8.478211115013991},
 {'id': 215, 'name': 'Jeff Buckley', 'score': 8.20765548699943},
 {'id': 418, 'name': 'Sigur Rós', 'score': 8.165749068759201},
 {'id': 181, 'name': 'Paradise Lost', 'score': 7.79305917680151}]

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd

path = ""
db = pd.read_csv(path+"predictions.csv")
names = pd.read_csv(path+"Artists.dat", sep="\t")

## Initialize app
app = Flask(__name__)


### Endpoint - one route /ratings/top - one HTTP verb = POST
@app.route("/ratings/top", methods=["POST"])
def recommended_artists():
    ## read the body of the API call
    content = request.get_json()
    
    ## Interpretation of body
    if "userId" in content and type(content["userId"]) == int:
        userId = content["userId"]
    else:
        return "'userId' is required and should be an Integer."
        sys.exit("'userId' is required and should be an Integer.")
        
    if "count" in content and type(content["count"]) == int:
        count = content["count"]
    else:
        count = 5
    
    # Dunno why, without this step 'userId' turns out to be undefined
    uid = userId
    
    
    # Assumes a format with columns names as artistId, rows as users' ratings, with the first column being the userId
    # Get the user's predictions, sorted. Only keeps n (count) artists best ranked.
    ratings = db[db.iloc[:,0]==uid].iloc[0].sort_values(ascending=False).reset_index()[1:count+1]

    # The artist IDs got turned into string inthe process
    ratings.iloc[:,0] = ratings.iloc[:,0].astype(int)

    # Fetches the artist names using their IDs
    artist_names = [names[names.iloc[:,0] == ratings.iloc[i,0] ].iloc[0, 1] for i in range(len(ratings))]
    
    results = [{"id": int(ratings.iloc[i, 0]), # ID retrieved in the ratings dataframe, the int() avoid np.int32 format which cant' be jsonified
      "name": artist_names[i], # Name retrieved in the artist names list defined above
      "score": ratings.iloc[i, 1]} # Score retrieved in the ratings dataframe
     for i in range(count)]
    
    return jsonify(results)

### Put endpoint online
if __name__ == '__main__':
    app.run(host='localhost', port=6000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:6000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Feb/2019 21:14:15] "POST /ratings/top HTTP/1.1" 200 -
